# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [2]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
# It comes with the tensorflow-datasets module, therefore, if you haven't please install the package using
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data

That's where we load and preprocess our data.

In [3]:
# remember the comment from above
# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

# tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
# in our case, we are interesteed in the MNIST; the name of the dataset is the only mandatory argument
# there are other arguments we can specify, which we can find useful
# mnist_dataset = tfds.load(name='mnist', as_supervised=True)
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 

# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
# this is also where we make use of mnist_info (we don't have to count the observations)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# let's cast this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# let's also store the number of test samples in a dedicated variable (instead of using the mnist_info one)
num_test_samples = mnist_info.splits['test'].num_examples
# once more, we'd prefer an integer (rather than the default float)
num_test_samples = tf.cast(num_test_samples, tf.int64)


# normally, we would like to scale our data in some way to make the result more numerically stable
# in this case we will simply prefer to have inputs between 0 and 1
# let's define a function called: scale, that will take an MNIST image and its label
def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255.

    return image, label


# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_data = mnist_test.map(scale)


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
BUFFER_SIZE = 10000
shuffled_train_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

BATCH_SIZE = 90
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))

In [5]:
#Model
input_size = 28*28
output_size = 10
hidden_layer_size = 400
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape = (28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size,activation='sigmoid'),
                             tf.keras.layers.Dense(hidden_layer_size,activation='softmax')
])

In [6]:
#selecting optimizers and loss functions
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
NO_OF_EPOCHS = 50
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)
model.fit(train_data,epochs=NO_OF_EPOCHS,callbacks =[early_stopping,] ,
          validation_data=(validation_inputs,validation_targets),verbose=2)
#at 10th epoch val_loss is increasing

Epoch 1/50
600/600 - 14s - loss: 0.3729 - accuracy: 0.9003 - val_loss: 0.1409 - val_accuracy: 0.9567 - 14s/epoch - 24ms/step
Epoch 2/50
600/600 - 9s - loss: 0.1171 - accuracy: 0.9645 - val_loss: 0.0866 - val_accuracy: 0.9737 - 9s/epoch - 15ms/step
Epoch 3/50
600/600 - 9s - loss: 0.0749 - accuracy: 0.9774 - val_loss: 0.0646 - val_accuracy: 0.9822 - 9s/epoch - 15ms/step
Epoch 4/50
600/600 - 10s - loss: 0.0548 - accuracy: 0.9834 - val_loss: 0.0536 - val_accuracy: 0.9823 - 10s/epoch - 16ms/step
Epoch 5/50
600/600 - 9s - loss: 0.0384 - accuracy: 0.9882 - val_loss: 0.0454 - val_accuracy: 0.9858 - 9s/epoch - 15ms/step
Epoch 6/50
600/600 - 9s - loss: 0.0317 - accuracy: 0.9904 - val_loss: 0.0320 - val_accuracy: 0.9875 - 9s/epoch - 14ms/step
Epoch 7/50
600/600 - 10s - loss: 0.0242 - accuracy: 0.9927 - val_loss: 0.0290 - val_accuracy: 0.9912 - 10s/epoch - 16ms/step
Epoch 8/50
600/600 - 9s - loss: 0.0176 - accuracy: 0.9945 - val_loss: 0.0174 - val_accuracy: 0.9950 - 9s/epoch - 15ms/step
Epoch 9/50

In [10]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step - loss: 0.0732 - accuracy: 0.9814


In [11]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 98.14%


Epoch 1/10
540/540 - 7s - loss: 0.5209 - accuracy: 0.8605 - 7s/epoch - 14ms/step
Epoch 2/10
540/540 - 3s - loss: 0.2030 - accuracy: 0.9405 - 3s/epoch - 6ms/step
Epoch 3/10
540/540 - 3s - loss: 0.1567 - accuracy: 0.9544 - 3s/epoch - 6ms/step
Epoch 4/10
540/540 - 3s - loss: 0.1293 - accuracy: 0.9619 - 3s/epoch - 6ms/step
Epoch 5/10
540/540 - 3s - loss: 0.1073 - accuracy: 0.9682 - 3s/epoch - 6ms/step
Epoch 6/10
540/540 - 6s - loss: 0.0933 - accuracy: 0.9719 - 6s/epoch - 10ms/step
Epoch 7/10
540/540 - 5s - loss: 0.0828 - accuracy: 0.9752 - 5s/epoch - 10ms/step
Epoch 8/10
540/540 - 5s - loss: 0.0736 - accuracy: 0.9775 - 5s/epoch - 9ms/step
Epoch 9/10
540/540 - 4s - loss: 0.0657 - accuracy: 0.9800 - 4s/epoch - 8ms/step
Epoch 10/10
540/540 - 5s - loss: 0.0598 - accuracy: 0.9813 - 5s/epoch - 10ms/step